In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.formula.api as smf
import statsmodels.api as sm
from firth import firthLogit

In [2]:
# Load data
df_choice = pd.read_csv('intertemporal_choice_obs.csv',sep=',',index_col=0)

# Create dummy variables
pid_dummies = pd.get_dummies(df_choice['pid'], prefix='pid')
rw_dummies = pd.get_dummies(df_choice['b_fixed_rw'], prefix='factor_fixed_rw')
delay_dummies = pd.get_dummies(df_choice['b_delay'], prefix='factor_delay')
df_choice = pd.concat([df_choice,pid_dummies,rw_dummies,delay_dummies],axis=1)

cols_pid = [i for i in df_choice.columns if 'pid_' in i and i!= 'pid_1']

# Create interaction terms
factor_cols = [col for col in df_choice.columns if col.startswith('factor_fixed_rw') or col.startswith('factor_delay')]
for factor_col in factor_cols:
    factor_name = factor_col.replace('factor_', '')
    interaction_col = f'I_vary_{factor_name}'
    df_choice[interaction_col] = df_choice[factor_col] * df_choice['b_vary_rw']

# Covert boolean vairables to numerical variables
bool_cols = df_choice.select_dtypes(include=['bool']).columns
df_choice[bool_cols] = df_choice[bool_cols].astype(int)

# Create regression sample for Immed_Rw_Vary
x_cols1 = ['a_rw','b_vary_rw',
           'factor_fixed_rw_7','factor_fixed_rw_9','factor_delay_9','factor_delay_18',
           'I_vary_fixed_rw_7','I_vary_fixed_rw_9','I_vary_delay_9','I_vary_delay_18']

df_choice_immed = df_choice[df_choice['cond'] == 'Immed_Rw_Vary']
y1 = df_choice_immed['choice']
X1 = sm.add_constant(df_choice_immed[x_cols1 + cols_pid])

# Create sample for Delayed_Rw_Vary
x_cols2 = ['a_rw','b_vary_rw',
           'factor_fixed_rw_7','factor_fixed_rw_9',
           'I_vary_fixed_rw_7','I_vary_fixed_rw_9']

df_choice_delayed = df_choice[df_choice['cond'] == 'Delayed_Rw_Vary']
y2 = df_choice_delayed['choice']
X2 = sm.add_constant(df_choice_delayed[x_cols2 + cols_pid])

In [3]:
firth_reg_1 = firthLogit(y1,X1)
firth_reg_1.fit()

iteration: 0 , LL= 12751.908641337992
iteration: 1 , LL= 5848.334232711649
iteration: 2 , LL= 4266.579633452862
iteration: 3 , LL= 3606.98158435943
iteration: 4 , LL= 3389.758368902345
iteration: 5 , LL= 3353.1689375316205
iteration: 6 , LL= 3351.5572946751117
iteration: 7 , LL= 3351.5500972578798
iteration: 8 , LL= 3351.5500932456757
iteration: 9 , LL= 3351.5500932438845
iteration: 10 , LL= 3351.550093243884


In [4]:
wald_result_1 = firth_reg_1.wald()
wald_coef_result_1 = wald_result_1[wald_result_1['var_name'].isin(x_cols1)]
wald_coef_result_1.to_csv('firth_result_immed.csv')

Confidence level:  0.95


In [5]:
df_choice_immed_q = df_choice_immed[['a_rw','b_fixed_rw','b_vary_rw','b_delay']]
df_choice_immed_q['pred_firth'] = firth_reg_1.predict()
pred_firth_immed = df_choice_immed_q.groupby(['a_rw','b_fixed_rw','b_vary_rw','b_delay']).mean('pred_firth')
pred_firth_immed.reset_index().to_csv('firth_pred_immed.csv')

C:\Users\zarkwang\AppData\Local\Temp\ipykernel_1368\2395919629.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_choice_immed_q['pred_firth'] = firth_reg_1.predict()


In [6]:
firth_reg_2 = firthLogit(y2,X2)
firth_reg_2.fit()


iteration: 0 , LL= 6293.7021291342935
iteration: 1 , LL= 2618.0610249924903
iteration: 2 , LL= 1691.1154516442043
iteration: 3 , LL= 1224.3363985323476
iteration: 4 , LL= 992.4025445968265
iteration: 5 , LL= 899.3022986420766
iteration: 6 , LL= 876.6519023539877
iteration: 7 , LL= 874.5239159401027
iteration: 8 , LL= 874.4760096565784
iteration: 9 , LL= 874.4756705870433
iteration: 10 , LL= 874.4756687663141
iteration: 11 , LL= 874.475668756779
iteration: 12 , LL= 874.475668756729
iteration: 13 , LL= 874.4756687567289


In [7]:
wald_result_2 = firth_reg_2.wald()
wald_coef_result_2 = wald_result_2[wald_result_2['var_name'].isin(x_cols2)]
wald_coef_result_2.to_csv('firth_result_delayed.csv')

Confidence level:  0.95


In [8]:
df_choice_delayed_q = df_choice_delayed[['a_rw','b_fixed_rw','b_vary_rw','b_delay']]
df_choice_delayed_q['pred_firth'] = firth_reg_2.predict()
pred_firth_delayed = df_choice_delayed_q.groupby(['a_rw','b_fixed_rw','b_vary_rw','b_delay']).mean('pred_firth')
pred_firth_delayed.reset_index().to_csv('firth_pred_delayed.csv')

C:\Users\zarkwang\AppData\Local\Temp\ipykernel_1368\2143971205.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_choice_delayed_q['pred_firth'] = firth_reg_2.predict()
